In [1]:
from pathlib import Path

import typer
from loguru import logger
from tqdm import tqdm
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder

import scipy.stats as stats
from scipy.stats import zscore

from seaborn import histplot
import matplotlib.pyplot as plt
import seaborn as sns

from customer_churn_pridiction.config import PROCESSED_DATA_DIR, RAW_DATA_DIR


2024-10-28 01:48:36.135 | INFO     | customer_churn_pridiction.config:<module>:11 - PROJ_ROOT path is: /Users/saadkhalid/Documents/epita/s2/ai_methodology/customer-churn-prediction-pipeline


In [2]:
df_master = pd.read_excel(RAW_DATA_DIR / "E_Commerce_Dataset.xlsx", sheet_name="E Comm")
df = df_master.copy()

In [3]:
df.drop(columns=["CustomerID"], inplace=True)

In [4]:
df.describe()

,Churn,Tenure,CityTier,WarehouseToHome,HourSpendOnApp,NumberOfDeviceRegistered,SatisfactionScore,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
count,5630.000000,5366.000000,5630.000000,5379.000000,5375.000000,5630.000000,5630.000000,5630.000000,5630.000000,5365.000000,5374.000000,5372.000000,5323.000000,5630.000000
mean,0.168384,10.189899,1.654707,15.639896,2.931535,3.688988,3.066785,4.214032,0.284902,15.707922,1.751023,3.008004,4.543491,177.223030
std,0.374240,8.557241,0.915389,8.531475,0.721926,1.023999,1.380194,2.583586,0.451408,3.675485,1.894621,2.939680,3.654433,49.207036
min,0.000000,0.000000,1.000000,5.000000,0.000000,1.000000,1.000000,1.000000,0.000000,11.000000,0.000000,1.000000,0.000000,0.000000
25%,0.000000,2.000000,1.000000,9.000000,2.000000,3.000000,2.000000,2.000000,0.000000,13.000000,1.000000,1.000000,2.000000,145.770000
50%,0.000000,9.000000,1.000000,14.000000,3.000000,4.000000,3.000000,3.000000,0.000000,15.000000,1.000000,2.000000,3.000000,163.280000
75%,0.000000,16.000000,3.000000,20.000000,3.000000,4.000000,4.000000,6.000000,1.000000,18.000000,2.000000,3.000000,7.000000,196.392500
max,1.000000,61.000000,3.000000,127.000000,5.000000,6.000000,5.000000,22.000000,1.000000,26.000000,16.000000,16.000000,46.000000,324.990000


In [5]:
columns_to_fill = ["Tenure", "WarehouseToHome", "HourSpendOnApp", "OrderAmountHikeFromlastYear", 
                   "CouponUsed", "OrderCount", "DaySinceLastOrder"]

for column in columns_to_fill:
    mode_value = df[column].mode()[0]
    df[column].fillna(mode_value, inplace=True)

/var/folders/29/httfxy392318fgqt38j1wr7c0000gp/T/ipykernel_99206/3608640504.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[column].fillna(mode_value, inplace=True)


In [6]:
df.isnull().sum()

Churn                          0
Tenure                         0
PreferredLoginDevice           0
CityTier                       0
WarehouseToHome                0
PreferredPaymentMode           0
Gender                         0
HourSpendOnApp                 0
NumberOfDeviceRegistered       0
PreferedOrderCat               0
SatisfactionScore              0
MaritalStatus                  0
NumberOfAddress                0
Complain                       0
OrderAmountHikeFromlastYear    0
CouponUsed                     0
OrderCount                     0
DaySinceLastOrder              0
CashbackAmount                 0
dtype: int64

In [35]:
numeric_cols = ['Churn', 'Tenure', 'CityTier', 'WarehouseToHome', 'HourSpendOnApp', 'NumberOfDeviceRegistered', 'SatisfactionScore', 'NumberOfAddress', 'Complain', 'OrderCount', 'OrderAmountHikeFromlastYear', 'CouponUsed', 'DaySinceLastOrder', 'CashbackAmount']

z_scores = df[numeric_cols].apply(zscore)
outliers_mask = (abs(z_scores) > 3.0)
outliers = df[outliers_mask.any(axis=1)]

df_cleaned = df[~outliers_mask.any(axis=1)]

In [33]:
selected_features = ['Tenure','Complain', 'DaySinceLastOrder', 'CashbackAmount', 'SatisfactionScore']

In [37]:
df_cleaned.to_csv(PROCESSED_DATA_DIR / "cleaned_data.csv", index=False)